# Banking Data Assessment

Create a new warehouse and dashboard for the banking dataset.

The banking data is currently held locally in csv format: `raw_banking.csv`.

It has 10,000 rows of data.

## Docker Compose

Using Docker Compose, the data is moved into a PostgreSQL database.

In addition, containers for PGAdmin and Metabase are also created.

- PGAdmin - management tool of Postgres.
- Metabase - business intelligence, dashboards, and data visualization tools.

See `docker_compose_warehouse` folder for config files.

![docker_compose](./screenshots/docker_compose.png)

Data can now be viewed in PGAdmin, inside a Postgres database:

![pgadmin4](./screenshots/pgadmin.png)

## Data Build Tool (DBT)

DBT is a way of managing the transformation of data inside a data warehouse - it's the 'T' in ETL.

To install into the venv:

`pip install dbt-postgres`

Create the following config files inside your directory (see relevant files for more details):

- `dbt_project.yml`
- `profiles.yml` (kept separate for security reasons)

Run `dbt debug` to verify installation and connection to the warehouse.

![dbt_debug](./screenshots/dbt_debug_connection.png)

## The Assessment

With DBT now setup, I can now consider what insights the business wishes to get from the data.

Create a dashboard to answer the following questions about the data:

1. How much money does the bank have? Add all customer balances.
1. How much money do our customers make, check the estimate salary field.
1. Most popular card.
1. Figure out how to identify richer or poorer customers and which card type they have.
1. Credit score, balance, satisfaction by country.
1. Explore the data yourself and try to add some metrics and graphs of your own.

### Data Modelling

First, I will separate the data into facts and dimensions tables from the raw dataset. This helps organise the data and is also an opportunity to remove any erroneus entries, such as null values and duplicates.

I will then use these to create summary tables with which to build a metrics dashboard that answers the above questions.

- `facts`: quantitative data points that represent the business metrics, such as revenue, profit, units sold, or any other key performance indicators.
- `dimensions`: the descriptive attributes of the data, such as time, geography, product, customer, or any other relevant aspect of the business.

#### dimensions

- customers: `customerid`, `surname`, `geography`, `gender`, `age`, `tenure`, `estimatedsalary`, `isactivemember`, `exited`, `complain`, `satisfactionscore`, `numofproducts`, `hascrcard`, `creditscore`

#### facts

- accounts: `cardtype`, `balance`, `pointearned`
- join on `customerid`

In [ ]:
--customers.sql--

{{ config(
    materialized = 'table',
)}}

SELECT
    customerid,
    surname,
    gender,
    age,
    geography,
    tenure,
    estimatedsalary,
    isactivemember,
    exited,
    complain,
    satisfactionscore,
    numofproducts,
    hascrcard,
    creditscore
FROM {{ source("raw", "raw_banking") }}

In [ ]:
--accounts.sql--

{{ config(
    materialized = 'table',
)}}

SELECT
    cardtype,
    balance,
    pointearned,
    customerid
FROM {{ source("raw", "raw_banking") }}

![customers](./screenshots/customers.png)

![accounts](./screenshots/accounts.png)

### Summary Tables

Now, I will create a few summary tables by aggregating facts in order to gain insights into the data. 

These tables will be designed  to answer the client questions.

1. How much money does the bank have? Add all customer balances.

In [ ]:
SELECT SUM(balance) AS total_balance 
FROM {{ ref('accounts') }}

![total_balance](./screenshots/total_balance.png)

2. How much money do our customers make, check the estimate salary field.

In [ ]:
SELECT SUM(estimatedsalary) AS total_estimated_salaries
FROM {{ ref('customers') }}

![total_salaries](./screenshots/total_salaries.png)

3. Most popular card.

In [ ]:
SELECT 
    COUNT(*) AS amount_of_cardtype,
    cardtype
FROM {{ ref('accounts') }}
GROUP BY cardtype

![most_popular_cardtype](./screenshots/popular_cardtype.png)

4. Figure out how to identify richer or poorer customers and which card type they have.

In [ ]:
SELECT 
    accounts.cardtype,
    ROUND(AVG(customers.estimatedsalary), 2) AS avg_customer_salary
FROM {{ ref('accounts') }}
JOIN {{ ref('customers') }} ON accounts.customerid = customers.customerid
GROUP BY cardtype

![cardtype_vs_salary](./screenshots/cardtype_avg_salary.png)

5. Credit score, balance, satisfaction by country.

In [ ]:
SELECT 
    SUM(accounts.balance) AS total_balance,
    customers.geography,
    ROUND(AVG(customers.creditscore), 2) AS avg_credit_score,
    ROUND(AVG(customers.satisfactionscore), 4) AS avg_satisfactionscore
FROM {{ ref('accounts') }}
JOIN {{ ref('customers') }} ON accounts.customerid = customers.customerid
GROUP BY customers.geography

![creditscore_balance_ctry](./screenshots/creditscore.png)

Explore the data yourself and try to add some metrics and graphs of your own.

- customer satisfaction score vs number of products

In [ ]:
SELECT 
    numofproducts,
    ROUND(AVG(satisfactionscore), 1) AS avg_satisfactionscore
FROM {{ ref('customers') }}
GROUP BY numofproducts

![satisfaction](./screenshots/satisfaction.png)

- number of customers and total balance per country

In [ ]:
SELECT 
    SUM(accounts.balance) AS total_balance,
    customers.geography AS country,
    COUNT(customers) AS customer_count
FROM {{ ref('accounts') }}
JOIN {{ ref('customers') }} ON accounts.customerid = customers.customerid
GROUP BY customers.geography

![customers_by_ctry](./screenshots/customer_by_ctry.png)

- customer's who complained vs those who closed their accounts vs avg age

In [ ]:
SELECT 
    ROUND(AVG(age), 1) AS avg_age,
    complain, 
    exited
FROM {{ ref('customers') }}
GROUP BY complain, exited

![complaints](./screenshots/complaints.png)

## Metrics Dashboard

Metrics dashboard created using Metabase:

![bank_dashboard](./screenshots/bank_dashboard2.png)